In [ ]:
import scanpy as sc

sample = 'sample_id'

sample_adata= sc.read_10x_mtx(f'{data_dir}/{sample}/filtered_feature_bc_matrix', var_names='gene_symbols', cache=False)

#### preprocess the data for clustering as soupX input
adata_pp = sample_adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")
soupx_groups = adata_pp.obs["soupx_groups"]

del adata_pp
#############################

cells = sample_adata.obs_names
genes = sample_adata.var_names
data = sample_adata.X.T
##### read in the raw files 
sample_raw = sc.read_10x_mtx(f'{data_dir}/{sample}/raw_feature_bc_matrix', var_names='gene_symbols', cache=False)
sample_raw.var_names_make_unique()
data_tod = sample_raw.X.T


In [ ]:
import logging

import anndata2ri
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
library(SoupX)

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
sample_adata.layers["counts"] = sample_adata.X
sample_adata.layers["soupX_counts"] = out.T
sample_adata.X = sample_adata.layers["soupX_counts"]

In [ ]:
sample_adata.X = sample_adata.layers["soupX_counts"]
sc.pp.filter_genes(sample_adata, min_cells=20)
sample_adata.X = sample_adata.layers["counts"]
sample_adata.write_h5ad(os.path.join(results_dir,'sample_adata_soupX_counts.h5ad'))

### repeat the process for all samples